### Automated Sample Collection for Diffusion Testing

Use this notebook to interact with the Opentrons OT2 using the Advanced Control option on the robotic platform.

In [ ]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from opentrons import simulate, execute, protocol_api
from Prepare import OT2_Hcell_commands as HC

import pandas as pd
import time 
import numpy as np

import string

In [ ]:
#if needed
#import importlib
#importlib.reload(HC)

Paths may vary depending on file organization. All custom hardware must be available in the Custom Labware folder.

In [ ]:
labware_dir_path = r"./Custom Labware/"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

In [ ]:
path = "./H_cell_protocol.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)

## Experiment

The following cells are used to execute the workflow. To test the workflow without physically executing it on the OT2, you can replace the `execute.get_protocol_api()` function with `simulate.get_protocol_api()`. 

Note: warnings may appear while running the following cell. As long as the pipettes with tips have clearance over the tops of the dual H-cell assemblies (check before executing), this should not be a problem.

In [ ]:
protocol = execute.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan, well_order='row')

The following cell allows you to record any metadata with the log files output from the experiment. H1, H2, H3, etc refer to H-cell 1, H-cell 2, H-cell 3, etc. If using all 8 spots, add H5, H6, H7, H8 to the list.

In [ ]:
#if using 8 h-cells, add sample names for H5, H6, H7, H8

log_metadata={
    'author': 'Author Name',
    'sample_name':'Analyte Name',
    'hcell_membrane':'H1: sample_1_name, H2: sample_2_name, H3: sample_3_name, H4: sample_4_name',
    'solvent':'water',
    'restock_solvent':'water'
}

The following cell is where you define the experimental parameters.

Aliquot volume: volume of extraction from each half-cell, to be mixed with the sample_dilution_volume in the sample well plate. Maximum sample volume is 300 uL for a standard flat-bottom 96-well plate.

For the `HC.get_time_schedule(66, [66], [6])` function, the order is `(total experiment time, [length of variable delay A, B], [variable time delay between samples A, B])`. A 66 hour experiment, in which samples are collected every 6 hours for the entire experiment time would be written as `(66,[66],[6])`. Alternatively, the experimental time could be split into different sample frequencies. For example: an 84-hour experiment in which samples are collected every 3 hours for the first 12 hours, then every 6 hours for the next 12 hours, and every 12 hours until the end of the experiment would be `(96,[12,12,60],[3,6,12])`.

In [ ]:
aliquot_volume= 20 # uL
sample_dilution_volume = 180 #uL
time_schedule, delay_time = HC.get_time_schedule(66, [66], [6]) # in hrs

h_cell, wellplates= HC.get_Hcell_labware(loaded_dict)
h_cell_name , h_cell_dict = HC.get_hcell_name(h_cell)

sample_schedule = HC.get_sample_schedule(h_cell_name, wellplates, time_schedule, delay_time)

Running the following cell will begin the actual workflow execution. The only variables that may change between runs are `max_stock_vol` and `log_filename=''`. 

`max_stock_vol` is the amount of restock solution (water) in the 20-mL scintillation vials, in uL. It is recommended to report a lower volume than is actually present (i.e., 17000 uL rather than 20000 uL) so that the robot will move to the secondary vial before the primary vial is completely emptied. Since the pipette tip aspirates at 5 mm from the bottom of the vial, this will prevent potentially pipetting air instead of solvent.

`log_filename` is what the output log file will be saved as.

In [ ]:
HC.H_cell_protocol(protocol, loaded_dict,
                    sample_schedule, h_cell_dict, aliquot_volume=aliquot_volume,
                    sample_dilution_volume=sample_dilution_volume,
                    max_stock_vol= 17000, log_file_info= log_metadata, log_filename= 'name_of_experiment')